In [6]:
# -----------------------------
# 0. Dependências
# -----------------------------
import os
import json
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from sklearn.metrics import accuracy_score

# -----------------------------
# 1. Carregar Dataset
# -----------------------------
os.makedirs("../datasets", exist_ok=True)
dataset = Planetoid(root="../datasets/Cora", name="Cora")
data = dataset[0]

# -----------------------------
# 2. Definir Modelo GCN
# -----------------------------
class NodeClassifier(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(NodeClassifier, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return x

# -----------------------------
# 3. Funções de treino e teste
# -----------------------------
def train(gcn, optimizer, data):
    gcn.train()
    optimizer.zero_grad()
    out = gcn(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

def test(gcn, data):
    gcn.eval()
    out = gcn(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        acc = accuracy_score(data.y[mask].cpu(), pred[mask].cpu())
        accs.append(acc)
    return accs  # [train_acc, val_acc, test_acc]

# -----------------------------
# 4. Treinar modelo
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gcn = NodeClassifier(dataset.num_features, 16, dataset.num_classes).to(device)
data = data.to(device)

optimizer = torch.optim.Adam(gcn.parameters(), 
    lr=0.01, weight_decay=5e-4
)

for epoch in range(1, 201):
    loss = train(gcn, optimizer, data)
    if epoch % 20 == 0:
        train_acc, val_acc, test_acc = test(gcn, data)
        print(f"Epoch {epoch:03d}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}")

# -----------------------------
# 5. Salvar Modelo 
# -----------------------------
import os, json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

os.makedirs("../models", exist_ok=True)
ckpt_gcn = "../models/gcn_state.pt"
ckpt_meta  = "../models/meta.pt"
ckpt_graph = "../models/graph_artifacts.pt"

# Salva pesos
torch.save(gcn.state_dict(), ckpt_gcn)

# Salva metadados
torch.save({
    "num_features": dataset.num_features,
    "hidden_channels": 16,
    "num_classes": dataset.num_classes
}, ckpt_meta)

# Salva artefatos do grafo
torch.save({
    "x": data.x.cpu(),
    "edge_index": data.edge_index.cpu()
}, ckpt_graph)

print("\n")

# -----------------------------
# 6. Métricas adicionais 
# -----------------------------
gcn.eval()

with torch.no_grad():
    out = gcn(data.x, data.edge_index)
    pred = out.argmax(dim=1).cpu().numpy()
    labels = data.y.cpu().numpy()

    # Val
    val_idx = data.val_mask.cpu().numpy()
    val_acc  = accuracy_score(labels[val_idx], pred[val_idx])
    val_prec = precision_score(labels[val_idx], pred[val_idx], average="macro", zero_division=0)
    val_rec  = recall_score(labels[val_idx], pred[val_idx], average="macro", zero_division=0)
    val_f1   = f1_score(labels[val_idx], pred[val_idx], average="macro", zero_division=0)

    # Test
    test_idx = data.test_mask.cpu().numpy()
    test_acc  = accuracy_score(labels[test_idx], pred[test_idx])
    test_prec = precision_score(labels[test_idx], pred[test_idx], average="macro", zero_division=0)
    test_rec  = recall_score(labels[test_idx], pred[test_idx], average="macro", zero_division=0)
    test_f1   = f1_score(labels[test_idx], pred[test_idx], average="macro", zero_division=0)

# Monta dicionário e salva em JSON
results = {
    "val": {
        "accuracy": float(val_acc),
        "precision": float(val_prec),
        "recall": float(val_rec),
        "f1": float(val_f1),
    },
    "test": {
        "accuracy": float(test_acc),
        "precision": float(test_prec),
        "recall": float(test_rec),
        "f1": float(test_f1),
    }
}

os.makedirs("../results", exist_ok=True)
with open("../results/result.json", "w") as f:
    json.dump(results, f, indent=2)

print("\n")

Epoch 020, Loss: 0.2581, Train Acc: 1.0000, Val Acc: 0.7920, Test Acc: 0.8160
Epoch 040, Loss: 0.0548, Train Acc: 1.0000, Val Acc: 0.7720, Test Acc: 0.7970
Epoch 060, Loss: 0.0339, Train Acc: 1.0000, Val Acc: 0.7660, Test Acc: 0.7890
Epoch 080, Loss: 0.0429, Train Acc: 1.0000, Val Acc: 0.7680, Test Acc: 0.8000
Epoch 100, Loss: 0.0420, Train Acc: 1.0000, Val Acc: 0.7720, Test Acc: 0.7970
Epoch 120, Loss: 0.0344, Train Acc: 1.0000, Val Acc: 0.7660, Test Acc: 0.7960
Epoch 140, Loss: 0.0267, Train Acc: 1.0000, Val Acc: 0.7660, Test Acc: 0.7960
Epoch 160, Loss: 0.0257, Train Acc: 1.0000, Val Acc: 0.7640, Test Acc: 0.7930
Epoch 180, Loss: 0.0221, Train Acc: 1.0000, Val Acc: 0.7660, Test Acc: 0.7970
Epoch 200, Loss: 0.0317, Train Acc: 1.0000, Val Acc: 0.7680, Test Acc: 0.7990




